In [1]:
!pip install google -q

In [2]:
import numpy as np
import pandas as pd
import os
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

PATH_TO_DATA_FOLDER = "../input/fake-news-task/"
TRAIN_PARSING_FOLDER = "../input/kontyptrain/"
TEST_PARSING_FOLDER = "../input/kontyp-test/"

train = pd.read_csv(os.path.join(PATH_TO_DATA_FOLDER,'train.tsv'), sep = '\t')
test = pd.read_csv(os.path.join(PATH_TO_DATA_FOLDER,'test.tsv'), sep = '\t')


# Создание словаря заголовков для парсинга во время первого запуска для train
# np.save('to_query', {title : None for title in train.title.values})

# Создание словаря заголовков для парсинга во время первого запуска для test
# np.save('to_query_test', {title : None for title in test.title.values})

Гугл будет банить данный парсер достаточно быстро, что будет вызывать к падению тредов. Поэтому производится сохранение уже полученной информации, и перезапуск ноутбука на kaggle/colab для получения машины с новым ip.

Для того, чтоб парсер не падал достаточно запустить его в 1 треде.

In [3]:
from threading import Thread
import numpy as np
from googlesearch import search
PANORAMA_LINKS = ["panorama", "ryb.ru", "kolibri.press"]
PAUSE_SECONDS = 5

def is_panorama_title(query):
    query = '"' + query + '"'
    for link in search(query, tld='ru', lang = 'ru', stop=10, pause=PAUSE_SECONDS):
        if any([panorama_link in link for panorama_link in PANORAMA_LINKS]):
            return True
    return False

def query_job(subset, thread):
    for i, query in enumerate(subset):
        if i % 10 == 0:
            print(f'Completed {i} in Thread {thread}')
        to_query[query] = is_panorama_title(query)
            
    print(f'Finished Thread {thread}')

def split(a, n):
    k, m = divmod(len(a), n)
    return [a[i * k + min(i, m) : (i + 1) * k + min(i + 1, m)] for i in range(n)]

def main(num_threads, num_queries):
    # create threads
    work = list()
    subset_len = num_queries * num_threads
    subset = [k for k, v in to_query.items() if v is None][:subset_len]
    splitted = split(subset, num_threads)
    for item in splitted:
        work.append(item)
    threads = [Thread(target=query_job, args=(job, i)) for i, job in enumerate(work)]

    # start the threads
    for thread in threads:
        thread.start()

    # wait for the threads to complete
    for thread in threads:
        thread.join()

if __name__ == "__main__":
    to_query = np.load(os.path.join(TRAIN_PARSING_FOLDER,'to_query.npy'), allow_pickle=True).item()

    try:
        main(40, 120)
    except Exception:
        pass
    
    np.save('to_query', to_query)
    print(len([k for k,v in to_query.items() if v is not None]))

Finished Thread 0Finished Thread 1

Finished Thread 2
Finished Thread 3Finished Thread 4
Finished Thread 5

Finished Thread 6
Finished Thread 7
Finished Thread 8
Finished Thread 9
Finished Thread 10
Finished Thread 11Finished Thread 12

Finished Thread 13
Finished Thread 14
Finished Thread 15Finished Thread 16
Finished Thread 17

Finished Thread 18
Finished Thread 19
Finished Thread 20
Finished Thread 21
Finished Thread 22
Finished Thread 23
Finished Thread 24
Finished Thread 25Finished Thread 26
Finished Thread 27
Finished Thread 28
Finished Thread 29
Finished Thread 30

Finished Thread 31
Finished Thread 32
Finished Thread 33
Finished Thread 34Finished Thread 35
Finished Thread 36

Finished Thread 37
Finished Thread 38Finished Thread 39

5757


In [4]:
train['preds'] = train['title'].apply(lambda x: 1 if to_query[x] else 0)

Посчитаем метрику f1

In [5]:
f1_score(train.is_fake, train.preds)

0.9998262984193156

Посмотрим на ошибочные значения

In [6]:
confusion_matrix(train.is_fake, train.preds)

array([[2879,    0],
       [   1, 2878]])

In [7]:
train.loc[train.is_fake != train.preds].values

array([['Рамзана Кадырова выдвинули на Нобелевскую премию мира', 1, 0]],
      dtype=object)

Результат 1 False Negative с вирусным заголовком

Получим результат работы скрипта для test

In [8]:
to_query = np.load(os.path.join(TEST_PARSING_FOLDER,'to_query_test.npy'), allow_pickle=True).item()

try:
    main(40, 120)
except Exception:
    pass

np.save('to_query_test', to_query)
print(len([k for k,v in to_query.items() if v is not None]))

Finished Thread 0
Finished Thread 1
Finished Thread 2
Finished Thread 3
Finished Thread 4
Finished Thread 5
Finished Thread 6
Finished Thread 7
Finished Thread 8
Finished Thread 9
Finished Thread 10
Finished Thread 11
Finished Thread 12Finished Thread 13
Finished Thread 14

Finished Thread 15
Finished Thread 16
Finished Thread 17
Finished Thread 18Finished Thread 19
Finished Thread 20
Finished Thread 21
Finished Thread 22
Finished Thread 23

Finished Thread 24
Finished Thread 25
Finished Thread 26
Finished Thread 27
Finished Thread 28Finished Thread 29
Finished Thread 30
Finished Thread 31
Finished Thread 32
Finished Thread 33

Finished Thread 34
Finished Thread 35Finished Thread 36

Finished Thread 37
Finished Thread 38
Finished Thread 39
1000


In [9]:
test_query = np.load(os.path.join(TEST_PARSING_FOLDER,'to_query_test.npy'), allow_pickle=True).item()

In [10]:
test['is_fake'] = test.title.apply(lambda x: 1 if test_query[x] else 0)

In [11]:
test.to_csv("predictions.tsv", sep = '\t')